In [1]:
import logging

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig

In [2]:
from azureml.core import Workspace

ws = Workspace.from_config()

WARNING - Warning: Falling back to use azure cli login credentials.
If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


In [24]:
from azureml.core import Datastore

sql_datastore = Datastore.get(workspace=ws, datastore_name="ado_sql_datastore")

In [35]:
from azureml.core import Dataset

improvements_sql_ds = Dataset.Tabular.from_sql_query((sql_datastore, 'SELECT * FROM Improvements'))

improvements_sql_ds.register(workspace=ws,
                          name="ai_ag_ado_improvements",
                          description = "Improvements from Azure DevOps")

2019-12-18 23:29:55.755816 | ActivityCompleted: Activity=from_sql_query, HowEnded=Failure, Duration=440.46 [ms], Info = {'activity_id': 'b4e4b72f-4317-43e3-929d-25f8b0afab58', 'activity_name': 'from_sql_query', 'activity_type': 'PublicApi', 'app_name': 'TabularDataset', 'source': 'azureml.dataset', 'version': '1.0.76', 'completionStatus': 'Success', 'durationMs': 2.5}, Exception=DatasetValidationError; Cannot load any data from the datastore using the SQL query "{}". Please make sure the datastore and query is correct.


DatasetValidationError: Cannot load any data from the datastore using the SQL query "{}". Please make sure the datastore and query is correct.

In [34]:
from azureml.core import Dataset

feedback_sql_ds = Dataset.Tabular.from_sql_query((sql_datastore, 'SELECT * FROM FeedbackItems'))

feedback_sql_ds.register(workspace=ws,
                          name="ai_ag_ado_feedack",
                          description = "Feedback from Azure DevOps")

{
  "source": null,
  "definition": [
    "ReadDatastoreSql"
  ],
  "registration": {
    "id": "e718c0cc-341a-4cba-b2a9-fbaf54bea4b7",
    "name": "ai_ag_ado_feedack",
    "version": 1,
    "description": "Feedback from Azure DevOps",
    "workspace": "Workspace.create(name='dciborowws', subscription_id='0ca618d2-22a8-413a-96d0-0f1b531129c3', resource_group='dciborow-ado-test')"
  }
}